# Bug - 1D wave equation blowing up

This ipython notebook reproduces the bug discussed in `github` issue [Quazartech/DG_Maxwell Issue #6](https://github.com/QuazarTech/DG_Maxwell/issues/6)

While evolving the wave equation using forward Euler method, The amplitude of the wave function (which should be a conserved quantity) inceasases with time and eventually blows up exponentially.


>### Note

- Each function will be shown along with its corresponding test function in a cell after discussing the theory
- Use the following command to get the documentation of a function
    ```
        <function>?
    ```

In [1]:
#Required libraries are imported and plot parameters are set in this cell
import subprocess

from matplotlib import pyplot as plt
import numpy as np
from scipy import integrate
from scipy import special as sp
from tqdm import trange
import arrayfire as af
af.set_backend('opencl')
af.set_device(1)
plt.rcParams['figure.figsize'] = 9.6, 6.
plt.rcParams['figure.dpi'] = 100
plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'serif'
plt.rcParams['text.usetex'] = True
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['axes.titlesize'] = 'medium'
plt.rcParams['axes.labelsize'] = 'medium'
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['xtick.major.pad'] = 8
plt.rcParams['xtick.minor.pad'] = 8
plt.rcParams['xtick.color'] = 'k'
plt.rcParams['xtick.labelsize'] = 'medium'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.minor.size'] = 4
plt.rcParams['ytick.major.pad'] = 8
plt.rcParams['ytick.minor.pad'] = 8
plt.rcParams['ytick.color'] = 'k'
plt.rcParams['ytick.labelsize'] = 'medium'
plt.rcParams['ytick.direction'] = 'in'


In [7]:
#Utility functions used in the code
def multiply(a, b):
    '''
    '''
    return a * b

def power(a, b):
    '''
    '''
    return a ** b
    
def linspace(start, end, number_of_points):
    '''
    Linspace implementation using arrayfire.
    '''
    X = af.range(number_of_points, dtype = af.Dtype.f64)
    d = (end - start) / (number_of_points - 1)
    X = X * d
    X = X + start

    return X

# Theory and functions used

The 1D wave equation is given by the equation

\begin{align}
\frac{\partial u(x, t)}{\partial t} + \frac{\partial F(u)}{\partial x} = 0
\end{align}

The solutions for the wave equation will be found for each of the
elements separately. For each element, the $x$ space is mapped to
$\xi$ space to and the calculations are done in the $\xi$
space. A $\xi$ has a domain from $\xi \epsilon [-1, 1]$.

This $\xi$ space is divided into n Legendre-Gauss-Lobatto (LGL)
points which are the roots of the equation

\begin{align}
    (1 - \xi^2)P_{n - 1}'(\xi) = 0
\end{align}

Where $P_n(\xi)$ is the Legendre polynomial with index n.

The function to obtain the LGL points along with a test function for the same
are in the cell below.

In [8]:
#This cell contains the function which returns LGL points
#It is tested with values obtained from http://mathworld.wolfram.com/LobattoQuadrature.html

def LGL_points(N):
    """
    Calculates and returns the LGL points for a given N, These are roots of
    the formula.
    :math: `(1 - \\xi^2) P_{n - 1}'(\\xi) = 0`
    Legendre polynomials satisfy the recurrence relation
    :math: `(1 - x^2) P_n' (x) = -n x P_n(x) + n P_{n - 1} (x)`
    
    Parameters
    ----------
    N : Int
        Number of LGL nodes required
    
    Returns
    -------
    lgl : arrayfire.Array [N 1 1 1]
          An arrayfire array consisting of the Lagrange-Gauss-Lobatto Nodes.
    
    Reference
    ---------
    http://mathworld.wolfram.com/LobattoQuadrature.html
    """
    xi                 = np.poly1d([1, 0])
    legendre_N_minus_1 = N * (xi * sp.legendre(N - 1) - sp.legendre(N))
    lgl_nodes          = legendre_N_minus_1.r
    lgl_nodes.sort()
    lgl_nodes          = af.interop.np_to_af_array(lgl_nodes)
    
    return lgl_nodes

def test_LGL_points():
    '''
    Comparing the LGL nodes obtained by LGL_points with
    the reference nodes for N = 6
    '''
    reference_nodes  = af.np_to_af_array(np.array([-1., -0.7650553239294647,\
                                  -0.28523151648064504, 0.28523151648064504,\
                                        0.7650553239294647, 1.]))
    calculated_nodes = (LGL_points(6))
    assert(af.max(af.abs(reference_nodes - calculated_nodes)) <= 1e-14)

An element is defined using two points. One of the points, say $x_i$
is mapped to $-1$ in the $\xi$ space and the second point,
say $x_{i + 1}$ is mapped to $1$ in the $\xi$ space. Using this
information we can map all the points in the $x$ space to $\xi$
space with the formula:

\begin{align} \label{eq:isopara_map}
    x = \frac{(1 - \xi)}{2} x_i + \frac{(1 + \xi)}{2} x_{i + 1}
\end{align}

The function to map a given $xi$ to $x$ space along with a test function in the cell below.

In [9]:
def mapping_xi_to_x(x_nodes, xi):
    '''
    Parameters
    ----------

    x_nodes : arrayfire.Array
                Element nodes.

    xi      : numpy.float64
                Value of :math: `\\xi`coordinate for which the corresponding
                :math: `x` coordinate is to be found.
    
    Returns
    -------
    x : arrayfire.Array
        :math: `x` value in the element corresponding to :math:`\\xi`.
    '''
    
    N_0 = (1 - xi) / 2
    N_1 = (1 + xi) / 2

    N0_x0 = af.bcast.broadcast(multiply, N_0, x_nodes[0])
    N1_x1 = af.bcast.broadcast(multiply, N_1, x_nodes[1])
    x     = N0_x0 + N1_x1
    
    return x

def test_mapping_xi_to_x():
    '''
    A test function to check the mappingXiToX function in wave_equation module,
    The test involves passing trial element nodes and :math: `\\xi` and
    comparing it with the x obatined by passing the trial parameters to
    mappingXiToX function.
    '''
    threshold = 1e-14
    
    test_element_nodes = af.interop.np_to_af_array(np.array([7, 11]))
    test_xi            = 0
    analytical_x_value = 9
    numerical_x_value  = mapping_xi_to_x(test_element_nodes, test_xi)

    assert af.abs(analytical_x_value - numerical_x_value) <= threshold

Lagrange Basis polynomials calculated over the LGL points is used in the code.
The Lagrange basis polynomial $L_p(\xi)$ is taken as
\begin{align}
    L_p(\xi) =
    \prod_{m = 0, m \neq p}^{m = N - 1} \frac{\xi - \xi_m}{\xi_m - \xi_p}
\end{align}

The function to calculate the Lagrange basis polynomials calculated using N LGL points
is shown in the cell below

In [10]:
def lagrange_basis_coeffs(x):    
    '''
    A function to get the coefficients of the Lagrange basis polynomials for
    a given set of x nodes.

    This function calculates the Lagrange basis
    polynomials by this formula:
    :math::
    `L_i = \\prod_{m = 0, m \\notin i}^{N - 1}\\frac{(x - x_m)}{(x_i - x_m)}`

    Parameters
    ----------
    x : numpy.array
        A numpy array consisting of the :math: `x` nodes using which the
        lagrange basis functions need to be evaluated.

    Returns
    -------
    lagrange_basis_poly : numpy.ndarray
                          A :math: `N \\times N` matrix containing the
                          coefficients of the Lagrange basis polynomials such
                          that :math:`i^{th}` lagrange polynomial will be the
                          :math:`i^{th}` row of the matrix.
    
    [NOTE] : Loops are used since calculation of Lagrange basis functions is
             done only once in the code.
    '''
    X = np.array(x)
    lagrange_poly_coeffs = np.zeros([X.shape[0], X.shape[0]])

    for j in np.arange(X.shape[0]):
        lagrange_basis_j = np.poly1d([1])
        
        for m in np.arange(X.shape[0]):
            if m != j:
                lagrange_basis_j *= np.poly1d([1, -X[m]]) \
                                    / (X[j] - X[m])
        lagrange_poly_coeffs[j] = lagrange_basis_j.c
    
    lagrange_poly_coeffs = af.np_to_af_array(lagrange_poly_coeffs)
    
    return lagrange_poly_coeffs

def test_lagrange_basis_coeffs():
    '''
    Function to test the lBasisArray function in global_variables module by
    passing 8 LGL points and comparing the numerically obtained basis function
    coefficients to analytically calculated ones. This array of the coefficients
    is required later on in the program.

    Reference
    ---------
    The link to the sage worksheet where the calculations were carried out.

    https://cocalc.com/projects/1b7f404c-87ba-40d0-816c-2eba17466aa8/files
    /PM_2_5/wave_equation/worksheets/l_basis_array.sagews
    '''
    threshold = 1e-12
    reference_basis_coeffs = np.zeros([8, 8])

    reference_basis_coeffs = np.array([[-3.351562500008004,\
    3.351562500008006, 3.867187500010295, -3.867187500010297,\
    - 1.054687500002225, 1.054687500002225, 0.03906249999993106,\
                                        - 0.03906249999993102],
    [8.140722718246403, - 7.096594831382852, - 11.34747768400062,\
    9.89205188146461, 3.331608712119162, - 2.904297073479968,\
    - 0.1248537463649464, 0.1088400233982081],
    [-10.35813682892759, 6.128911440984293, 18.68335515838398,\
    - 11.05494463699297, - 8.670037141196786, 5.130062549476987,\
    0.3448188117404021, - 0.2040293534683072],
    [11.38981374849497, - 2.383879109609436, - 24.03296250200938,\
    5.030080255538657, 15.67350804691132, - 3.28045297599924,\
    - 3.030359293396907, 0.6342518300700298],
    [-11.38981374849497, - 2.383879109609437, 24.03296250200939,\
    5.030080255538648, - 15.67350804691132, - 3.28045297599924,\
    3.030359293396907, 0.6342518300700299], 
    [10.35813682892759, 6.128911440984293, -18.68335515838398,\
    - 11.05494463699297, 8.670037141196786, 5.130062549476987,\
    - 0.3448188117404021, - 0.2040293534683072],
    [-8.140722718246403, - 7.096594831382852, 11.34747768400062,\
    9.89205188146461, -3.331608712119162, - 2.904297073479968,\
    0.1248537463649464, 0.1088400233982081], 
    [3.351562500008004, 3.351562500008005, - 3.867187500010295,\
    - 3.867187500010298, 1.054687500002225, 1.054687500002224, \
    - 0.039062499999931, - 0.03906249999993102]])
                
    reference_basis_coeffs  = af.np_to_af_array(reference_basis_coeffs)
    calculated_basis_coeffs = lagrange_basis_coeffs(LGL_points(8))
    assert af.max(af.abs(reference_basis_coeffs - calculated_basis_coeffs)) < 1e-10

def lagrange_basis_value(N_LGL):
    '''
    A function which calculates and returns the value of the Lagrange basis
    polynomials obtained using LGL points, at the LGL points.
    
    Parameters
    ----------
    N_LGL : int
            Number of LGL points into which the :math:`\\xi` space is to be
            split.
    
    Returns
    -------
    Li_xi : arrayfire.Array [N N 1 1]
            An array consisting of the value of the lagrange basis polynomials
            evaluated at the LGL points. :math: `L_i(\\xi_k)` is arranged in a
            2D matrix with 'i' increasing along the rows and k across columns.
    '''
    xi_array   = af.transpose(LGL_points(N_LGL))
    index      = af.flip(af.range(N_LGL))
    xi_power   = af.broadcast(power, xi_array, index)
    
    basis_coeffs_array = lagrange_basis_coeffs(LGL_points(N_LGL))
    
    Li_xi = af.matmul(basis_coeffs_array, xi_power)
    
    return Li_xi

def test_lagrange_basis_value():
    '''
    The Lagrange basis polynomials :math: `L_i(\\xi_k)` will be zero when
    'i' is not equal to 'k' and is 1 in all other cases. An identity matrix
    is expected
    '''
    threshold = 1e-13
    calculated_lagrange_basis_value = lagrange_basis_value(8)
    analytical_lagrange_basis_value = af.identity(8, 8, dtype=af.Dtype.f64)
    assert af.max(af.abs(calculated_lagrange_basis_value - analytical_lagrange_basis_value)) <= threshold

Integration is performed throughout the code using Gauss Lobatto  $\href{https://en.wikipedia.org/wiki/Gaussian_quadrature#Gauss.E2.80.93Lobatto_rules}{\mathrm{quadrature}}$.
It is a method to find integral of the type,

\begin{align}
    \int_{-1}^{1}f(\xi) d\xi
\end{align}

According to Lobatto quadrature, the integral is given by


\begin{align}
    \int_{-1}^{1}f(\xi) d\xi
    = \frac{2}{N (N - 1)}[f(1) + f(-1)] + \sum_{i = 1}^{N - 1}w_i f(x_i)
\end{align}

where,
$x_i$ are the solution of $L'_{N - 1}$, the LGL points.
$w_i$ are the weights calculated at $x_i$, it is given by,

\begin{align}
    w_i = \frac{2}{N(N - 1)[P'_{n - 1}(x_i)]^2}
\end{align}

Function to calculate the Gauss-Lobatto weight and its corresponding
test function are in the cell below.


In [11]:
def lobatto_weight_function(n = 8, xi_LGL = LGL_points(8)):
    '''
    Calculates and returns the weight function for an index :math:`n`
    and points :math: `x`.
    
    :math::
        w_{n} = \\frac{2 P(x)^2}{n (n - 1)},
        Where P(x) is $ (n - 1)^th $ index.
    
    Parameters
    ----------
    n      : int
             Index for which lobatto weight function
    
    xi_LGL : arrayfire.Array
             1D array of points where weight function is to be calculated.
    
    
    Returns
    -------
    gaussLobattoWeights : arrayfire.Array
                          An array of lobatto weight functions for
                          the given :math: `x` points and index.
    Reference
    ---------
    Gauss-Lobatto weights Wikipedia link-
    https://en.wikipedia.org/wiki/
    Gaussian_quadrature#Gauss.E2.80.93Lobatto_rules
    
    '''
    P = sp.legendre(n - 1)
    
    gaussLobattoWeights = (2 / (n * (n - 1)) / (P(xi_LGL))**2)
    gaussLobattoWeights = af.np_to_af_array(gaussLobattoWeights)
    
    return gaussLobattoWeights

def test_lobatto_weight_function():
    '''
    Compares the Lobatto weights obtained by the function and reference values
    '''
    threshold = 1e-1
    N_LGL = 6
    lobatto_weights = lobatto_weight_function(N_LGL, LGL_points(N_LGL))
    analytical_weights = af.Array([0.06666666666666667, 0.378474956297847,\
                                  0.5548583770354863, 0.5548583770354863,\
                                   0.378474956297847, 0.06666666666666667])
    
    assert(af.max(af.abs(lobatto_weights - analytical_weights)) <= threshold)

The cell below initializes the parameters required for time evolution of the wave equation

In [12]:
x_nodes    = af.interop.np_to_af_array\
             (np.array([-1., 1.]))               #The domain of the function which is of interest.
N_LGL      = 8                                   #The number of LGL points into which an element is split.
xi_LGL     = LGL_points(N_LGL)                   #Array containing the LGL points in xi space
    
l_basis_coeffs = lagrange_basis_coeffs\
                     (LGL_points(N_LGL))         #An array containing the coefficients of Lagrange polynomials
                                                 #evaluated over LGL points(Used later)
l_basis_value  = lagrange_basis_value(N_LGL)     #An array of value of Lagrange basis polynomials
                                                 #evaluated over LGL points
N_Elements   = 10                                #Number of elements the domain is to be divided into.
element_size = af.sum((x_nodes[1] -
               x_nodes[0]) / N_Elements)         #Size of equally divided elements

The strong form (differential form) of the wave equation is converted
to the weak form (integral form). To do this, A test function
$v(x)$ is multiplied with the wave equation and the resultant
expresion is integrated w.r.t. $x$.


\begin{align}
v(x)\frac{\partial u(x, t)}{\partial t}
&+ v(x)\frac{\partial F(u)}{\partial x}
= 0\\
\int v(x)\frac{\partial u(x, t)}{\partial t} dx
&+ \int v(x)\frac{\partial F(u)}{\partial x} dx
= 0 \\
\int v(x)\frac{\partial u(x, t)}{\partial t} dx
&+ v(x) F(u) - \int F(u)\frac{\partial v(x)}{\partial x} dx
= 0 
\end{align}

Taking $u^{n}(x)$ to be the wave function at the $n^{th}$
time step. The aim is to find $u^{n + 1}(x)$ (i.e. wave function at $(n + 1)^{th}$ time step).

Taking an increment of $\Delta t$ in time between the consecutive timesteps $n$ and $n + 1$

\begin{align}
    \frac{\int v(x)u^{n + 1}(x) dx - \int v(x)u^{n}(x) dx}
    {\Delta t} + v(x) F(u^n) - \int F(u^n)\frac{\partial v(x)}{\partial x} dx
    = 0
\end{align}

\begin{align}
    \int v(x)u^{n + 1}(x) dx &- \int v(x)u^{n}(x) dx + \Delta t \{v(x) F(u^n) - \int F(u^n)
    \frac{\partial v(x)}{\partial x} dx\}
    = 0  \\
    \int v(x)u^{n + 1}(x) dx &= \int v(x)u^{n}(x) dx - \Delta t \{v(x) F(u^n) - \int F(u^n)
    \frac{\partial v(x)}{\partial x} dx\}
\end{align}

Mapping $x$ into $\xi$ space (whose domain is [-1, 1]), for a single element,

\begin{align}
    \int^1_{-1} v(\xi)u^{n + 1}(\xi) \frac{dx}{d\xi} d\xi &= \int^1_{-1} v(\xi)u^{n}(\xi)
    \frac{dx}{d\xi}d\xi - \Delta t \{v(\xi) F(u^n) - \int^1_{-1} F(u^n)
    \frac{\partial v(\xi)}{\partial \xi} d\xi\}
\end{align}

Choosing $v(\xi)$ as the Lagrange basis $L_p$
polynomials created using the $N$ Lagrange-Gauss-Lobatto(LGL) points

\begin{align}
    L_p(\xi) =
    \prod_{m = 0, m \neq p}^{m = N - 1} \frac{\xi - \xi_m}{\xi_m - \xi_p}
\end{align}


For each element, the wave function can be written in $\xi$ space
as a linear combination of the Lagrange basis polynomials created
using the LGL points in that space, i.e.,

\begin{align}
    u^n(\xi) = \sum_{i = 0}^{N - 1} u^n_i L_i(\xi)
\end{align}

In the above equation, $u^n_i$ are the coefficients for $L_i$
and $N$ is the total number LGL points inside the $\xi$ space.

Substituting equation $u^n(\xi) = \sum_{i = 0}^{N - 1} u^n_i L_i(\xi)$ and $v(\xi) = L_p(\xi)$
into the modified weak form of the wave equation,

\begin{align}
    \sum_{i = 0}^{N - 1} u^{n + 1}_i \int L_p(\xi)L_i(\xi)
    \frac{dx}{d\xi} d\xi &=
    \sum_{i = 0}^{N - 1} u^n_i \int L_p(\xi)L_i(\xi)
    \frac{dx}{d\xi}d\xi - \Delta t \{L_p(\xi) F(u^n) - \int F(u^n)
    \frac{\partial L_p(\xi)}{\partial \xi} d\xi\}
\end{align}


\begin{align}
A_{pi} &= \int L_p(\xi)L_i(\xi) \frac{dx}{d\xi} d\xi \\
b_p    &= \sum_{i = 0}^{N - 1} u^n_i \int L_p(\xi)L_i(\xi)
\frac{dx}{d\xi}d\xi - \Delta t \{L_p(\xi) F(u^n) - \int F(u^n)
\frac{\partial L_p(\xi)}{\partial \xi} d\xi\}
\end{align}

On varying $p$ from $0$ to $N - 1$, $N$ such linear equations
for $N$ variable $u^{n + 1}_i$, $i \epsilon {0, 1, \cdots, N - 1}$.
Writing all the system of linear equations in matrix representation.

\begin{align}
\begin{bmatrix}
A_{00} & A_{01} & \cdots & A_{0{N-1}} \\
A_{10} & A_{11} & \cdots & A_{1{N-1}} \\
\vdots & \vdots & \ddots & \vdots     \\
A_{(N - 1)0} & A_{(N - 1)1} & \cdots & A_{(N - 1){(N-1)}}
\end{bmatrix}
\begin{bmatrix}
u^{n + 1}_0 \\
u^{n + 1}_1 \\
\vdots      \\
u^{n + 1}_{N - 1}
\end{bmatrix}
=
\begin{bmatrix}
b_0       \\
b_1       \\
\vdots    \\
b_{N - 1} \\
\end{bmatrix}
\end{align}

or

\begin{align}
Au^{n + 1} = b \\
u^{n + 1}  = A^{-1} b
\end{align}

## A Matrix
To find the solution of the wave equation in the next time step, the
$A$ matrix has to be calculated, which has elements $A_{pi}$ described as.
\begin{align}
    A_{p i} = \int_{-1}^{1} L_p(\xi)L_i(\xi) \frac{dx}{d\xi} d\xi
\end{align}

This integral can be evaluated using the Gauss-Lobatto Quadrature.

\begin{align}
    \sum_{k = 0}^{N - 1} w_k L_p(\xi_k)L_i(\xi_k) \frac{dx}{d\xi}|_{\xi_k}
\end{align}


Note that all the matrices will be treated as a $3$ dimensional matrix
with the dimensions being represented by $[d1, d2, d3$, where, $d1, d2$
are the dimensions corresponding to the row and column respectively.


\begin{align}
[L_p] &=
\begin{bmatrix}
L_0(\xi_0)       & L_0(\xi_1)       & \cdots & L_0(\xi_{N - 1})       \\
L_1(\xi_0)       & L_1(\xi_1)       & \cdots & L_1(\xi_{N - 1})       \\
\vdots           & \vdots           & \ddots & \vdots                 \\
L_{N - 1}(\xi_0) & L_{N - 1}(\xi_1) & \cdots & L_{N - 1}(\xi_{N - 1})
\end{bmatrix}_{[N, 1, N]}\\
[L_i] &=
\begin{bmatrix}
L_0(\xi_0)       & L_0(\xi_1)       & \cdots & L_0(\xi_{N - 1})       \\
L_1(\xi_0)       & L_1(\xi_1)       & \cdots & L_1(\xi_{N - 1})       \\
\vdots           & \vdots           & \ddots & \vdots                 \\
L_{N - 1}(\xi_0) & L_{N - 1}(\xi_1) & \cdots & L_{N - 1}(\xi_{N - 1})
\end{bmatrix}_{[1, N, N]}\\
[w_k] &=
\begin{bmatrix}
w_0 \\
w_1 \\
\vdots     \\
w_{N - 1}
\end{bmatrix}_{[1, 1, N]} \\
\Bigg[\Bigg (\frac{dx}{d\xi} \Bigg )_k \Bigg ]
&=
\begin{bmatrix}
\frac{dx}{d\xi}|_{\xi = \xi_0} \\
\frac{dx}{d\xi}|_{\xi = \xi_1} \\
\vdots     \\
\frac{dx}{d\xi}|_{\xi = \xi_{N - 1}}
\end{bmatrix}_{[1, 1, N]}
=
\begin{bmatrix}
\Big (\frac{dx}{d\xi} \Big ) \\
\Big (\frac{dx}{d\xi} \Big ) \\
\vdots     \\
\Big (\frac{dx}{d\xi} \Big )
\end{bmatrix}_{[1, 1, N]}
\end{align}

Tiling $[L_p]_{[N, 1, N]}$ across $2^{nd}$ dimension
and $[L_i]_{[1, N, N]}$ along $1^{st}$ dimension multiply corresponding elements

<img src = "Lp_Li_xi_k.png">
$3D$ array obtained by doing element wise multiplication of
tiled $[L_p]$ and $[L_i]$


Now, the matrices $[w_k]$ and
$\Bigg[\Bigg (\frac{dx}{d\xi} \Bigg )_k \Bigg ]$ from previous equations
are tiled along the $1^{st}$ and the $2^{nd}$ dimensions and multiplied element wise with the
3D array shown in the figure above. The resultant matrix is


<img src="Lp_Li_w_k_DxDxi_k.png">



To obtain the $A$ matrix, we add all the elements along the $3^{rd}$ dimension.
The resultant $A$ matrix is given by,

\begin{align}
A =
\begin{bmatrix}
\sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_0(\xi_k)L_0(\xi_k) & \sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_0(\xi_k)L_1(\xi_k) & \cdots & \sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_0(\xi_k)L_{N - 1}(\xi_k) \\
\sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_1(\xi_k)L_0(\xi_k) & \sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_1(\xi_k)L_1(\xi_k) & \cdots & \sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_1(\xi_k)L_{N - 1}(\xi_k) \\
\vdots & \vdots & \ddots & \vdots \\
\sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_{N - 1}(\xi_k)L_0(\xi_k) & \sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_{N - 1}(\xi_k)L_1(\xi_k) & \cdots & \sum_{k = 0}^{N - 1} w_k \Big(\frac{dx}{d\xi}\Big)_k L_{N - 1}(\xi_k)L_{N - 1}(\xi_k) \\
\end{bmatrix}
\end{align}
